<a href="https://colab.research.google.com/github/Tariquzzaman-faisal/VITD/blob/main/xlmroberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mouting to Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pwd

'/content'

# Importing

In [ ]:
!pip install transformers

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load pre-trained model and tokenizer


In [ ]:
!pip install sentencepiece

In [ ]:
!apt-get install cmake build-essential pkg-config libgoogle-perftools-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libgoogle-perftools-dev is already the newest version (2.9.1-0ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [ ]:
model_name = "xlm-roberta-base"
num_labels = 3

model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_data(data, tokenizer, max_length):
    input_ids = []
    attention_masks = []

    for text in data['sentence1']:
        inputs = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        input_ids.append(inputs["input_ids"])
        attention_masks.append(inputs["attention_mask"])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(data['label'])

    return input_ids, attention_masks, labels


In [ ]:
datasets_dir = '/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/Tariq/split/'

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
train_df = pd.read_csv(datasets_dir+'train.csv')
val_df = pd.read_csv(datasets_dir+'validation.csv')
test_df = pd.read_csv(datasets_dir+'test.csv')

In [ ]:
train_df

,sentence1,label
0,যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কি...,1
1,এই বিচার শেষ বিচার নয়<PUNC>আসল বিচার হবে আল্ল...,0
2,আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্...,2
3,দেশটা সুস্থ নাই,0
4,আপনার কথা দু<PUNC>খ জনক আগে বিডিও থাকলে কেন ধর...,0
...,...,...
2695,হল বন্ধ করার আগে নিউমার্কেট বন্ধ করা উচিৎ<PUNC...,2
2696,এ স্বাধীন দেশে ভোটটা অন্তত<PUNC> আমাদের দিতে দ...,0
2697,আল হামদুলিল্লাহ্<PUNC<PUNC>PUNC<PUNC><PUNC<PUN...,0
2698,এই গুলা ত বিচার করবায় ঐ তোমরা ত ইন্ডিয়ান না,0


In [ ]:
max_length = 512
train_input_ids, train_attention_masks, train_labels = tokenize_data(train_df, tokenizer, max_length)
val_input_ids, val_attention_masks, val_labels = tokenize_data(val_df, tokenizer, max_length)
test_input_ids, test_attention_masks, test_labels = tokenize_data(test_df, tokenizer, max_length)


In [ ]:
# Create DataLoader
batch_size = 16
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [ ]:
# Hyperparameters for early stopping
patience = 3  # Number of epochs to wait for improvement
min_delta = 0.001  # Minimum change in validation accuracy to be considered an improvement
best_val_accuracy = 0
epochs_without_improvement = 0

# Initialize the best_model_state
best_model_state = None

# Training loop
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 20  # Adjust as needed

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    train_preds = []
    train_true_labels = []

    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
        labels = batch[2]

        optimizer.zero_grad()
        outputs = model(**inputs)
        logits = outputs.logits

        loss = torch.nn.functional.cross_entropy(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        predicted_labels = torch.argmax(logits, dim=1)
        train_preds.extend(predicted_labels.cpu().numpy())
        train_true_labels.extend(labels.cpu().numpy())

    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = accuracy_score(train_true_labels, train_preds)

    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

    # Validation loop
    model.eval()
    val_preds = []
    val_true_labels = []

    with torch.no_grad():
        for batch in val_dataloader:
            batch = tuple(t.to(device) for t in batch)
            inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
            labels = batch[2]

            outputs = model(**inputs)
            logits = outputs.logits

            predicted_labels = torch.argmax(logits, dim=1)
            val_preds.extend(predicted_labels.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true_labels, val_preds)
    print(f"Epoch {epoch+1}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping check
    if val_accuracy > best_val_accuracy + min_delta:
        best_val_accuracy = val_accuracy
        epochs_without_improvement = 0
        # Save the current best model state
        best_model_state = model.state_dict()
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"Early stopping after {epoch+1} epochs.")
        # Load the best model state
        model.load_state_dict(best_model_state)
        break


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Train Loss: 0.9790, Train Accuracy: 0.5248
Epoch 1, Validation Accuracy: 0.6474
Epoch 2, Train Loss: 0.7869, Train Accuracy: 0.6548
Epoch 2, Validation Accuracy: 0.7489
Epoch 3, Train Loss: 0.6118, Train Accuracy: 0.7537
Epoch 3, Validation Accuracy: 0.7707
Epoch 4, Train Loss: 0.5119, Train Accuracy: 0.8144
Epoch 4, Validation Accuracy: 0.7549
Epoch 5, Train Loss: 0.4211, Train Accuracy: 0.8441
Epoch 5, Validation Accuracy: 0.7805
Epoch 6, Train Loss: 0.3197, Train Accuracy: 0.8837
Epoch 6, Validation Accuracy: 0.7398
Epoch 7, Train Loss: 0.2710, Train Accuracy: 0.9011
Epoch 7, Validation Accuracy: 0.7639
Epoch 8, Train Loss: 0.2012, Train Accuracy: 0.9304
Epoch 8, Validation Accuracy: 0.7564
Early stopping after 8 epochs.


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# # Testing loop
# model.eval()
# test_preds = []
# test_true_labels = []

# with torch.no_grad():
#     for batch in test_dataloader:
#         batch = tuple(t.to(device) for t in batch)
#         inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
#         labels = batch[2]

#         outputs = model(**inputs)
#         logits = outputs.logits

#         predicted_labels = torch.argmax(logits, dim=1)
#         test_preds.extend(predicted_labels.cpu().numpy())
#         test_true_labels.extend(labels.cpu().numpy())

# test_accuracy = accuracy_score(test_true_labels, test_preds)
# print(f"Test Accuracy: {test_accuracy:.4f}")

# # Generate classification report
# class_names = ["Non Violence", "Passive Violence", "Direct Violence"]  # Replace with your actual class names
# report = classification_report(test_true_labels, test_preds, target_names=class_names)
# print("Classification Report:")
# print(report)

In [ ]:
# # Create a DataFrame with index and prediction columns
# result_df = pd.DataFrame({
#     'index': range(len(test_true_labels)),
#     'prediction': test_preds
# })

# # Save the DataFrame to a CSV file
# result_df.to_csv('/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/models/xlmroberta/predictions.csv', index=False)

# Saving tuned model

In [ ]:
from transformers import AutoModel, AutoConfig

# Define the path to the saved model
model_path = "/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/models/xlmroberta/xlmroberta_trained_model.pth"

# Save the model
torch.save(model.state_dict(), model_path)

In [ ]:
# Create an instance of the model
model2 = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Load the saved parameters into the model
model2.load_state_dict(torch.load(model_path))

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
# Testing loop
model2.to(device)  # Move the model to the same device as tensors
model2.eval()
test_preds = []
test_true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
        labels = batch[2]

        outputs = model2(**inputs)
        logits = outputs.logits

        predicted_labels = torch.argmax(logits, dim=1)
        test_preds.extend(predicted_labels.cpu().numpy())
        test_true_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(test_true_labels, test_preds)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate classification report
class_names = ["Non Violence", "Passive Violence", "Direct Violence"]  # Replace with your actual class names
report = classification_report(test_true_labels, test_preds, target_names=class_names)
print("Classification Report:")
print(report)

Test Accuracy: 0.5595
Classification Report:
                  precision    recall  f1-score   support

    Non Violence       1.00      0.56      0.72      2016
Passive Violence       0.00      0.00      0.00         0
 Direct Violence       0.00      0.00      0.00         0

        accuracy                           0.56      2016
       macro avg       0.33      0.19      0.24      2016
    weighted avg       1.00      0.56      0.72      2016



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Create a DataFrame with index and prediction columns
result_df = pd.DataFrame({
    'index': range(len(test_true_labels)),
    'prediction': test_preds
})

# Save the DataFrame to a CSV file
result_df.to_csv('/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/models/xlmroberta/predictions.csv', index=False)